IMPORTS

In [ ]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

THESE ARE FILES THAT WOULD PROBABLY BE IMPORTED INTO A VOLUME

In [ ]:
demopref_data = pd.read_csv('demopref_data')
U_I_merged_df = pd.read_csv('U_I_merged_df')

THESE WILL PROBABLY BE AT JUST BEFORE THE LAST LAYER OF RECOMMENDATIONS

In [ ]:
# Load the user_item_matrix from file
user_item_matrix = pd.read_pickle('user_item_matrix.pkl')
# Loading the demopref kmeans model
with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

THESE ARE FUNCTIONS NECESSARY FOR THE BUILD OF THE RECOMMENDATIONS, SO WILL PROBABLY BE IN THE EARLIER LAYERS

In [ ]:

# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Define a function to get the K most similar users
def find_similar_users(user_id, user_similarity, k=10):
    # Get the index of the user
    user_index = user_item_matrix.index.tolist().index(user_id)

    # Get the similarity scores for all users (convert to list to sort it)

    similarity_scores = user_similarity[user_index]

    # Sort the users based on their similarity scores

    sorted_indices = similarity_scores.argsort()[::-1]

    # Get the K most similar users
    similar_users = [list(U_I_merged_df['id_student'].unique())[i] for i in sorted_indices if i != user_index][:k]
    return similar_users


In [ ]:

def recommend_courses(user_id, user_similarity, user_item_matrix, U_I_merged_df, k=10):
    # Get the K most similar users
    similar_users = find_similar_users(user_id, user_similarity, k=k)

    # Get the courses taken by similar users
    similar_courses = []
    for user in similar_users:
        user_courses = U_I_merged_df.loc[U_I_merged_df['id_student'] == user]['code_module']
        similar_courses.extend(user_courses)

    # Count the frequency of each course
    course_counts = Counter(similar_courses)
    print(course_counts)

    # Get the courses the user is enrolled in
    userT_courses = U_I_merged_df.loc[U_I_merged_df['id_student'] == user_id]['code_module']
    list_userT_courses = list(userT_courses)
    list_userT_courses = Counter(list_userT_courses)
    # Sort the courses by frequency and get the top K courses
    top_courses = [course for course, count in course_counts.most_common(k) if course not in list_userT_courses]

    return top_courses, list_userT_courses



    return top_courses

RECOMMENDATIONS BASED ON USER_ITEM_MATRIX

In [ ]:
recommended_courses, list_userT_courses = recommend_courses(205719, user_similarity, user_item_matrix, U_I_merged_df, k=3)
print(f"Courses enrolled in by user 205719: {list_userT_courses}")
print(f"Top recommended courses for user 205719: {recommended_courses}")

RECOMMEND COURSES BASED ON VLE PREFERENCES & CLUSTER OF TARGET USER


In [ ]:
# define the target student ID
target_student_id = 205719

# select the cluster label of the target student
target_student_cluster = demopref_data.loc[demopref_data['id_student'] == target_student_id, 'cluster_label'].values[0]

# find the similar students based on VLE preferences
similar_students = find_similar_users(target_student_id, user_similarity, k=15)

# filter the similar students based on their cluster label
similar_students = demopref_data.loc[demopref_data['cluster_label'] == target_student_cluster & demopref_data['id_student'].isin(similar_students)]

# Find the courses that are most frequently taken by the filtered similar students
freq_courses = similar_students.iloc[:, :7].sum().dropna().sort_values(ascending=False)
freq_courses = freq_courses[freq_courses != 0].nlargest(3)

# Recommend the courses that the target student has not taken yet but are among the most frequently taken courses by the filtered similar students
target_student_courses = demopref_data.loc[demopref_data['id_student'] == target_student_id].iloc[0, :7]
recommended_courses = freq_courses[~target_student_courses.astype(bool)].index.tolist()
print(f"Recommended courses for student {target_student_id}: {recommended_courses}")

## THE STEPS BELOW ARE PROBABLY RELEVANT MORE TO NIFI, PRE TREATEMENT AND DATA INGESTION AND PREPARATION AND THINGS LIKE THAT

THESE MIGHT BE STEPS OF PRE TREATEMENT BEFORE LOADING TO THE KMEANS MODEL

In [ ]:
loaded_data = pd.read_csv('df_full_cleaned_predicted_imdband')

demo_features = ['id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'studied_credits', 'disability']
perf_features = ['code_module', 'code_presentation', 'id_assignment', 'score', 'is_banked', 'num_of_prev_attempts', 'final_result']

additional_demo_data = pd.read_csv('studentInfo.csv')

added_features = additional_demo_data[['id_student', 'code_module', 'code_presentation', 'final_result', 'studied_credits']]

added_features = pd.get_dummies(added_features, columns=['code_module', 'code_presentation'], prefix=['code_module', 'code_presentation'])
# Merge the student information data with the original DataFrame
demopref_data = pd.merge(loaded_data, added_features, on=['id_student', 'code_presentation_2013B','code_presentation_2013J','code_presentation_2014B','code_presentation_2014J', 'code_module_AAA','code_module_BBB','code_module_CCC','code_module_DDD','code_module_EEE','code_module_FFF','code_module_GGG' ] )

# Drop the irrelevant columns
demopref_data.drop(['sum_click', 'id_site'], axis=1, inplace=True)
demopref_data = pd.get_dummies(demopref_data, columns=['final_result'], prefix='final_result')
demopref_data['id_student'] = demopref_data['id_student'].astype(int)
demopref_data['id_assessment'] = demopref_data['id_assessment'].astype(int)

THIS WAS THE CODE TO GENERATE THE USER_ITEM_MATRIX

In [ ]:
user_item_matrix = pd.pivot_table(U_I_merged_df, values='sum_click', index='id_student', columns='id_site', aggfunc='sum', fill_value=0)